In [1]:
from lib.db.models import create_tables
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///test.sqlite")
create_tables(engine)

In [5]:
from lib.db.models import get_or_create, Metric, Results
from sqlalchemy.orm import Session
from lib.db.models import EvaluationRun, EvaluationStatus, Experiment

def save_metrics(engine, exp: str, snapshot: str, metrics):

    # Add metrics if they don't exist
    with Session(engine) as session, session.begin():
        for metric_name in metrics.keys():
            get_or_create(engine, Metric, name=metric_name)

    with Session(engine) as session, session.begin():
        evaluation = session.query(EvaluationRun).filter(
            EvaluationRun.experiment.has(name=exp),
            EvaluationRun.snapshot == snapshot).one()
        evaluation_id = evaluation.id
        metric_id = {
            metric.name: metric.id
            for metric in session.query(Metric).filter(
                Metric.name.in_(metrics.keys())).all()
        }
        for metric_name, metric_value in metrics.items():
            result = Results(evaluation_id=evaluation_id,
                                  metric_id=metric_id[metric_name],
                                  value=metric_value)
            session.add(result)


with Session(engine) as session, session.begin():
    running_status = get_or_create(engine, EvaluationStatus, name='RUNNING')
    experiment = get_or_create(engine, Experiment, name='test1')

    running_eval = session.query(EvaluationRun).filter(EvaluationRun.experiment.has(name='test1'),
                                                            EvaluationRun.snapshot == 'test_snapshot',
                                                            EvaluationRun.status.has( name='RUNNING')).one_or_none()
    if running_eval is not None:
        pass
    else:
        new_eval = EvaluationRun(experiment=experiment, snapshot='test_snapshot', status=running_status)
        session.add(new_eval)
        
with Session(engine) as session, session.begin():
    running_status = get_or_create(engine, EvaluationStatus, name='FINISHED')
    running_eval = session.query(EvaluationRun).filter(EvaluationRun.experiment.has(name='test1'),
                                                            EvaluationRun.snapshot == 'test_snapshot',
                                                            EvaluationRun.status.has( name='RUNNING')).one()
    running_eval.status = running_status
    
# save_metrics(engine, 'test1', 'test_snapshot', {'x': 1, 'y': 2})

In [9]:
with Session(engine) as session, session.begin():
    running_eval = session.query(EvaluationRun).filter(EvaluationRun.experiment.has(name='test1'),
                                                            EvaluationRun.snapshot == 'test_snapshot',
                                                            EvaluationRun.status.has( name='FINISHED')).one_or_none()
    session.delete(running_eval)

In [7]:

with Session(engine) as session, session.begin():
    running_eval = session.query(EvaluationRun).filter(EvaluationRun.experiment.has(name='test1'),
                                                            EvaluationRun.snapshot == 'test_snapshot',
                                                            EvaluationRun.status.has( name='RUNNING')).one_or_none()
    print(running_eval.created_at, running_eval.updated_at)
# with Session(engine) as session, session.begin():
#     finished_status = get_or_create(engine, EvaluationStatus, name='FINISHED')
#     new_eval = session.query(EvaluationRun).filter(EvaluationRun.experiment.has(name='test1'),
#                                                         EvaluationRun.snapshot == 'test_snapshot').one()
#     new_eval.status = finished_status

2024-09-12 16:45:19.454368 2024-09-12 16:50:32.308349
